# CH06.다층 퍼셉트론 실습

## 실습 순서

0. 사용할 라이브러리와 패키지 불러오기

1.   (데이터) 데이터 정의하기
2. (모델) 다층 퍼셉트론 모델 불러오기
3. (모델 학습) 모델 학습하기
4. (모델 성능 평가) 모델 성능 평가하기

## [실습 6-1] scikit-learn 라이브러리를 이용한 다층 퍼셉트론



### 0. 사용할 라이브러리와 패키지 불러오기




In [2]:
# numpy를 각각 np로 재지정해 불러온다.
# scikit-learn 패키지 중 사용할 패키지를 불러온다.
import numpy as np
from sklearn.neural_network import MLPClassifier

### 1. 데이터 정의하기



In [3]:
# XOR 데이터를 정의한다.
# XOR은 두 입력값이 서로 같으면 0, 서로 다르면 1로 분류하는 논리 회로 게이트다.
X = [[0, 0], [0, 1], [1, 0], [1, 1]]
y = [0, 1, 1, 0]

### 2. 다층 퍼셉트론 모델 불러오기




In [4]:
# scikit-learn 라이브러리를 통해 구현되어 있는 다층 퍼셉트론 모델을 불러온다.
classifier = MLPClassifier(hidden_layer_sizes = (2, ), activation = 'logistic',
                          learning_rate_init = 0.1)

### 3. 모델 학습하기




In [5]:
# fit 함수를 통해 학습을 진행한다.
# fit 함수에는 1) 가중치 초기화 2) 손실 함수 계산 3) 가중치 업데이트가 모두 구현되어 있다.
classifier.fit(X, y)
w = classifier.coefs_
w_0 = classifier.intercepts_
print(w)
print(w_0)

[array([[ 0.20597759, -0.12359798],
       [-0.42928902,  0.02346334]]), array([[0.24910724],
       [0.19862848]])]
[array([-0.4721148 , -0.19532294]), array([-0.09976397])]


### 4. 모델 성능 평가하기




In [6]:
# predict 함수를 통해 다층 퍼셉트론으로 얻은 예측값을 추출한다.
y_pred = classifier.predict(X) # 테스트 데이터에 대하여 결과 예측
print(y_pred)

[1 1 1 1]


## [실습 6-2] 함수를 직접 정의해 구현한 다층 퍼셉트론




### 1. 데이터 정의하기



In [7]:
# XOR 데이터를 정의한다.
# XOR은 두 입력값이 서로 같으면 0, 서로 다르면 1로 분류하는 논리 회로 게이트다.
X = np.array([0, 0, 1, 1, 0, 1, 0, 1]).reshape(2, 4)
y = np.array([0, 1, 1, 0]).reshape(1, 4)
print(X)

[[0 0 1 1]
 [0 1 0 1]]


### 2. 다층 퍼셉트론 모델 불러오기





In [8]:
# 다층 퍼셉트론 모델 선언하기
# scikit-learn 라이브러리를 통해 구현되어 있는 다층 퍼셉트론 모델을 불러온다.
# 1) 가중치 초기화
# 2) 가중합 함수
# 3) 활성화 함수
# 4) 손실 함수
# 5) 가중치 업데이트

np.random.seed(59)
W_0 = np.random.rand(2, 2)
W_1 = np.random.rand(2, 1)
B_0 = np.random.random((1, 2))
B_1 = np.random.random((1, 1))
lr = 0.1

def weighted_sum(X, W, B):
    return np.dot(W.T, X) + B.T

def sigmoid(x):
    return 1 / (1+np.exp(-x))

def BCE_loss(y, y_hat):
    loss = np.mean(y * (np.log1p(y_hat)) + (1 - y) * np.log1p(1 - y_hat))
    return -loss

def gradient_update(X, H, y, y_hat):
    global W_0, B_0, W_1, B_1, lr
    dW_1 = np.dot(H, (y_hat - y).T)
    dB_1 = np.sum(y_hat - y, axis = 1, keepdims = True).T
    dH = np.dot(W_1, y_hat - y)

    dZ_1 = dH * (H * (1 - H))
    dW_0 = np.dot(X, dZ_1.T)
    dB_0 = np.sum(dZ_1, axis = 1, keepdims = True).T

    W_0 = W_0 - lr * dW_0
    W_1 = W_1 - lr * dW_1
    B_0 = B_0 - lr * dB_0
    B_1 = B_1 - lr * dB_1

### 3. 모델 학습하기




In [9]:
# 앞서 구현한 함수 다섯 개를 모두 사용하여 모델을 학습한다.
for epoch in range(1000):
		Z_1 = weighted_sum(X, W_0, B_0)
		H = sigmoid(Z_1)
		Z_2 = weighted_sum(H, W_1, B_1)
		y_hat = sigmoid(Z_2)

		loss = BCE_loss(y, y_hat)
		gradient_update(X, H, y, y_hat)

print(W_0)
print(W_1)
print(B_0)
print(B_1)

[[5.35774448 2.3563632 ]
 [5.32050553 2.3506189 ]]
[[ 5.87445068]
 [-5.51541982]]
[[-1.80781176 -3.43980731]]
[[-2.70355729]]


### 4. 모델 성능 평가하기



In [10]:
# predict 함수를 통해 다층 퍼셉트론으로 얻은 예측값을 추출한다.
def predict(X):
		global W_0, W_1, B_0, B_1
		Z_1 = weighted_sum(X, W_0, B_0)
		H = sigmoid(Z_1)
		Z_2 = weighted_sum(H, W_1, B_1)
		y_hat = sigmoid(Z_2).reshape(4)

		pred_class = []
		pred_class = [1 if i > 0.5 else 0 for i in y_hat]
		return np.array(pred_class)

y_pred = predict(X)
print(y_pred)

[0 1 1 0]
